In [ ]:
# Template Notebook cho Data Preparation - Milestone 1
# Thành viên: [TÊN] - Dataset: [TÊN DATASET]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

# Import utilities (sau khi đã tạo file)
import sys
sys.path.append('../src/')
from visualization_utils import create_standard_plots, plot_class_distribution, plot_split_distributions

# ==========================================
# 1. LOAD VÀ KHÁM PHÁ DỮ LIỆU
# ==========================================

print("="*50)
print("1. LOAD VÀ KHÁM PHÁ DỮ LIỆU")
print("="*50)

# TODO: Load dataset của bạn
# data = pd.read_csv('path/to/your/dataset.csv')

# Hiển thị thông tin cơ bản
print("Dataset shape:", data.shape)
print("\nDataset info:")
print(data.info())
print("\nFirst 5 rows:")
print(data.head())

# Kiểm tra missing values
print("\nMissing values:")
print(data.isnull().sum())

# Thống kê mô tả
print("\nDescriptive statistics:")
print(data.describe())

# ==========================================
# 2. DATA PREPROCESSING
# ==========================================

print("\n" + "="*50)
print("2. DATA PREPROCESSING")
print("="*50)

# TODO: Xử lý missing values nếu có
# data = data.dropna()  # hoặc fillna()

# TODO: Xử lý categorical features (nếu có)
# Ví dụ cho Palmer Penguins:
# categorical_cols = ['sex', 'island']
# data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# TODO: Chuẩn bị features và target
# X = data.drop('target_column', axis=1)
# y = data['target_column']

print("Features shape:", X.shape)
print("Target shape:", y.shape)

# ==========================================
# 3. VISUALIZE CLASS DISTRIBUTION
# ==========================================

print("\n" + "="*50)
print("3. VISUALIZE CLASS DISTRIBUTION")
print("="*50)

# Sử dụng function chuẩn
dataset_name = "[TÊN DATASET CỦA BạN]"  # Thay đổi tên này
target_col = "[TÊN CỘT TARGET]"        # Thay đổi tên này

plot_class_distribution(data, target_col, f'{dataset_name} - Original Data',
                       f'../visualizations/{dataset_name.lower().replace(" ", "_")}/original_distribution.png')

# ==========================================
# 4. CHIA DỮ LIỆU THEO 4 TỶ LỆ
# ==========================================

print("\n" + "="*50)
print("4. CHIA DỮ LIỆU THEO 4 TỶ LỆ")
print("="*50)

# Định nghĩa các tỷ lệ split
split_ratios = {
    '40/60': 0.4,
    '60/40': 0.6, 
    '80/20': 0.8,
    '90/10': 0.9
}

# Dictionary để lưu các split
data_splits = {}
target_splits = {}

# Thực hiện split cho từng tỷ lệ
for ratio_name, train_size in split_ratios.items():
    print(f"\nSplitting data with ratio {ratio_name}...")
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        train_size=train_size,
        stratify=y,  # Đảm bảo stratified split
        random_state=42,
        shuffle=True
    )
    
    # Lưu vào dictionary
    data_splits[ratio_name] = (X_train, X_test)
    target_splits[ratio_name] = (y_train, y_test)
    
    # In thông tin về split
    print(f"Train size: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
    print(f"Test size: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
    print(f"Train class distribution:\n{y_train.value_counts()}")
    print(f"Test class distribution:\n{y_test.value_counts()}")

# ==========================================
# 5. VISUALIZE SPLIT DISTRIBUTIONS  
# ==========================================

print("\n" + "="*50)
print("5. VISUALIZE SPLIT DISTRIBUTIONS")
print("="*50)

# Chuẩn bị data cho visualization
splits_for_viz = {}
for ratio_name in split_ratios.keys():
    y_train, y_test = target_splits[ratio_name]
    splits_for_viz[ratio_name] = (y_train, y_test)

# Vẽ biểu đồ so sánh
plot_split_distributions(splits_for_viz, dataset_name, 
                        f'../visualizations/{dataset_name.lower().replace(" ", "_")}/split_distributions.png')

# ==========================================
# 6. LƯU KẾT QUẢ
# ==========================================

print("\n" + "="*50)
print("6. LƯU KẾT QUẢ")
print("="*50)

# Tạo thư mục output
import os
output_dir = f'../results/{dataset_name.lower().replace(" ", "_")}'
os.makedirs(output_dir, exist_ok=True)

# Lưu các split để sử dụng trong Milestone 2
import pickle

# Lưu data splits
with open(f'{output_dir}/data_splits.pkl', 'wb') as f:
    pickle.dump({
        'data_splits': data_splits,
        'target_splits': target_splits,
        'original_data': (X, y),
        'dataset_info': {
            'name': dataset_name,
            'shape': data.shape,
            'features': list(X.columns),
            'target': target_col,
            'classes': list(y.unique())
        }
    }, f)

print(f"✅ Data splits saved to {output_dir}/data_splits.pkl")

# Tạo summary report
summary_stats = []
for ratio_name in split_ratios.keys():
    y_train, y_test = target_splits[ratio_name]
    summary_stats.append({
        'Split Ratio': ratio_name,
        'Train Size': len(y_train),
        'Test Size': len(y_test),
        'Train Class 0': (y_train == 0).sum() if 0 in y_train.values else 'N/A',
        'Train Class 1': (y_train == 1).sum() if 1 in y_train.values else 'N/A',
        'Test Class 0': (y_test == 0).sum() if 0 in y_test.values else 'N/A', 
        'Test Class 1': (y_test == 1).sum() if 1 in y_test.values else 'N/A'
    })

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv(f'{output_dir}/split_summary.csv', index=False)
print(f"✅ Summary saved to {output_dir}/split_summary.csv")

# Display summary
print("\nSplit Summary:")
print(summary_df)

print("\n" + "="*50)
print("MILESTONE 1 COMPLETED ✅")
print("="*50)
print(f"Dataset: {dataset_name}")
print(f"Original samples: {len(data)}")
print(f"Features: {X.shape[1]}")
print(f"Classes: {len(y.unique())}")
print(f"Splits created: {len(split_ratios)}")
print(f"Output directory: {output_dir}")
print("="*50)

# ==========================================
# 7. CHECKLIST CHO COORDINATOR
# ==========================================

print("\n📋 CHECKLIST CHO COORDINATOR:")
print("□ Dataset được load thành công")
print("□ Missing values được xử lý")
print("□ Categorical features được encode (nếu có)")
print("□ 4 tỷ lệ split được tạo với stratified sampling")
print("□ Class distributions được visualize")
print("□ Kết quả được lưu vào đúng thư mục")
print("□ Summary statistics được tạo")
print("\n🔄 Gửi kết quả này cho Coordinator để review!")